In [ ]:
from src.tools.datetime import format_timedelta

from src.projects.fagradalsfjall.common.paths import get_blog_post_subfolder
from src.projects.fagradalsfjall.common.project_settings import TS_PRIMARY_METRIC_DISPLAY_NAME
from src.projects.fagradalsfjall.common.model_eval_plot_curves import generate_all_rmse_plots
from src.projects.fagradalsfjall.common.model_eval_plot_simulations import plot_simulations
from src.projects.fagradalsfjall.common.model_repo import ModelRepo
from src.projects.fagradalsfjall.common.model_eval import ModelEvalResult, ValidationType

# 2. Visualization

In [ ]:
# -------------------------------------------------------------------------
#  Output path settings
# -------------------------------------------------------------------------
path_results_naive = get_blog_post_subfolder(3, "results_1_naive")
path_results_exp_decay = get_blog_post_subfolder(3, "results_2_exp_decay")

In [ ]:
# -------------------------------------------------------------------------
#  Load data
# -------------------------------------------------------------------------

# --- load results from model repo --------------------
naive_model_eval_results = ModelRepo.load_models(["naive-constant", "naive-mean"])
cv_model_eval_results = ModelRepo.load_models(["naive-exp-decay"])

# --- post-processing ---------------------------------
all_model_eval_results = naive_model_eval_results | cv_model_eval_results

model_exp_decay = all_model_eval_results["naive-exp-decay"].ts_model

In [ ]:
# -------------------------------------------------------------------------
#  PLOTS - NAIVE
# -------------------------------------------------------------------------
generate_all_rmse_plots(
    naive_model_eval_results, path_results_naive / "curves_with_threshold", plot_rmse_threshold=True
)
generate_all_rmse_plots(
    naive_model_eval_results, path_results_naive / "curves_without_threshold", plot_rmse_threshold=False
)

In [ ]:
# -------------------------------------------------------------------------
#  PLOTS - EXP. DECAY
# -------------------------------------------------------------------------
fig, ax = (
    model_exp_decay.cv.results.plot_1d("tau")
    .set_fig_title("Cross-validation - 'naive-exp-decay' - tau")
    .set_y_label(TS_PRIMARY_METRIC_DISPLAY_NAME)
    .set_x_label("tau\n(samples)")
    .create(w=12, h=8)
)
fig.savefig(path_results_exp_decay / "cv_results_tau.png", dpi=600)

In [ ]:
# -------------------------------------------------------------------------
#  PLOTS - ALL MODELS
# -------------------------------------------------------------------------
generate_all_rmse_plots(all_model_eval_results, path_results_exp_decay, highlight_models=["naive-exp-decay"])

In [ ]:
# -------------------------------------------------------------------------
#  Simulation results
# -------------------------------------------------------------------------
plot_simulations(naive_model_eval_results, path_results_naive)
plot_simulations(cv_model_eval_results, path_results_exp_decay)

In [ ]:
df_metrics = ModelEvalResult.all_metric_values_as_df(all_model_eval_results, ValidationType.TEST)

df_metrics["max_accurate_lead_time_samples"] = df_metrics["max_accurate_lead_time"].copy()
df_metrics["max_accurate_lead_time"] = df_metrics["max_accurate_lead_time"].apply(
    lambda x: format_timedelta(x * 15 * 60)
)

display(df_metrics)